In [1]:
from utils import *
from pathlib import Path
from typing import NamedTuple, Tuple
import pandas as pd
import boto3
from rekognition_objects import RekognitionFace

## Setup

In [2]:
val_df = get_valdf()
images = get_images()

In [3]:
class FaceAnalysis(NamedTuple):
    path: str
    age: Tuple[int, int]
    gender: str

In [4]:
rekognition_client = boto3.client('rekognition')

## Function for analyzing images

In [5]:
def aws_analyze_image(path: Path) -> FaceAnalysis:
    short_path = '/'.join(str(path).split('/')[-2:])
    try:
        with open(path, 'rb') as image_data:
            image = {'Bytes': image_data.read()}
            response = rekognition_client.detect_faces(Image = image, Attributes = ['ALL'])
            faces = [RekognitionFace(face) for face in response['FaceDetails']]
            face = faces[0]
            return FaceAnalysis(path=short_path, age=face.age_range, gender=face.gender)
    except:
        return FaceAnalysis(path=short_path, age=None, gender=None)

## Building the dataframe

In [6]:
paths, ages, genders = [], [], []

In [7]:
for idx, im in enumerate(images):
    if idx % 50 == 0:
        print(idx)
    face = aws_analyze_image(im)
    paths.append(face.path)
    ages.append(face.age)
    genders.append(face.gender)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
10050
10100
10150

In [8]:
analyzed_df = pd.DataFrame({'path': paths, 'predicted_age': ages, 'predicted_gender': genders})

In [9]:
analyzed_df['predicted_age'].isna().sum()

156

In [10]:
analyzed_df.head()

,path,predicted_age,predicted_gender
0,val/9733.jpg,"(23, 37)",Male
1,val/63.jpg,"(4, 14)",Female
2,val/6400.jpg,"(25, 39)",Female
3,val/823.jpg,"(26, 40)",Male
4,val/4217.jpg,"(2, 8)",Female


In [12]:
val_df.merge(analyzed_df, left_on = 'file', right_on = 'path').to_csv('aws_val_merged.csv')